# Fine tune GPTJ on SageMaker Training with Deepspeed

In this notebook we will finetune GPTJ on the processed [quotes dataset](https://www.kaggle.com/datasets/akmittal/quotes-dataset). This notebook/repo makes use of [this GitHub repo](https://github.com/Xirider/finetune-gpt2xl) where the Dockerfile has been adpated to be compliant with SageMaker. 

The dataset is in the format we wish to make inference with:

`<Catagory>: <AIGeneratedQuote>`+

Example: 
*We want to give GTJ a catagory and it must generate a quote*

`love: <AIGeneratedQuote>`

Take a look at the quote dataset we will be using in this notebook. 
1. [train.csv](https://raw.githubusercontent.com/marckarp/amazon-sagemaker-fine-tune-gptj/main/Finetune_GPTNEO_GPTJ6B/quotes_dataset/train.csv)
2. [validation.csv](https://raw.githubusercontent.com/marckarp/amazon-sagemaker-fine-tune-gptj/main/Finetune_GPTNEO_GPTJ6B/quotes_dataset/validation.csv)

If you wish to make use of your own dataset feel free to create a train and validation dataset with your own data to accomplish the task you are setting out to achieve. 

For more details on preparing a dataset please see [this link](https://github.com/mallorbc/Finetune_GPTNEO_GPTJ6B/tree/main/finetuning_repo#preparing-a-dataset).

In [3]:
!pip install sagemaker boto3 --upgrade

Looking in indexes: https://pypi.org/simple, https://pip.repos.neuron.amazonaws.com
  Using cached sagemaker-2.132.0.tar.gz (668 kB)
  Preparing metadata (setup.py) ... done
  Using cached boto3-1.26.67-py3-none-any.whl (132 kB)
  Using cached botocore-1.29.67-py3-none-any.whl (10.4 MB)
  Created wheel for sagemaker: filename=sagemaker-2.132.0-py2.py3-none-any.whl size=905449 sha256=bc553449c2403ff315c2a11f173aad2549b0ad504c2f65de13ffba3bd34c6bb3
  Stored in directory: /home/ec2-user/.cache/pip/wheels/b5/e9/08/e877c056814c01a15dc1b3de802cc212a254a50500fdc8ef66
Successfully built sagemaker
  Attempting uninstall: botocore
    Found existing installation: botocore 1.29.46
    Uninstalling botocore-1.29.46:
      Successfully uninstalled botocore-1.29.46
  Attempting uninstall: boto3
    Found existing installation: boto3 1.26.46
    Uninstalling boto3-1.26.46:
      Successfully uninstalled boto3-1.26.46
  Attempting uninstall: sagemaker
    Found existing installation: sagemaker 2.128.0

## Build & Push the container for SageMaker Training

In order to fine tune GPTJ we will have to make use of a docker container with Deepspeed installed. 
The Dockerfile is adapted from this repo [here](https://github.com/mallorbc/Finetune_GPTNEO_GPTJ6B/blob/main/Dockerfile). It has been adapted to be SageMaker compatible. 

Below we will define the deepspeed CLI command that will be run within our SageMaker Training Job. It has been paramterized using Enviroment variables so that we can have the ability to tune/customize the parameters when we kick of a SageMaker Training Job. 



In [4]:
%%writefile ../Finetune_GPTNEO_GPTJ6B/train
#!/bin/bash

df -h
cd finetuning_repo

deepspeed --num_gpus=$num_gpus run_clm.py --deepspeed $deepspeed --model_name_or_path EleutherAI/gpt-j-6B --train_file /opt/ml/input/data/train/train.csv --validation_file /opt/ml/input/data/validation/validation.csv --do_train --do_eval --fp16 --overwrite_cache --evaluation_strategy=$evaluation_strategy --output_dir $output_dir --num_train_epochs $num_train_epochs  --eval_steps $eval_steps --gradient_accumulation_steps $gradient_accumulation_steps --per_device_train_batch_size $per_device_train_batch_size --use_fast_tokenizer $use_fast_tokenizer --learning_rate $learning_rate --warmup_steps $warmup_steps --save_total_limit $save_total_limit --save_steps $save_steps --save_strategy $save_strategy --tokenizer_name $tokenizer_name --load_best_model_at_end=$load_best_model_at_end --block_size=$block_size --weight_decay=$weight_decay

Overwriting ../Finetune_GPTNEO_GPTJ6B/train


We set the train file we created above as executable. Once we have all our files ready we can build and push our image to ECR. 

In [5]:
%%sh 

cd ../Finetune_GPTNEO_GPTJ6B
chmod +x train


In [6]:
%%sh
cd ../Finetune_GPTNEO_GPTJ6B
./build_push_image.sh

Login Succeeded
Sending build context to Docker daemon  65.28MB
Step 1/20 : FROM nvidia/cuda:11.7.1-devel-ubuntu20.04
 ---> c4f8d3c928f4
Step 2/20 : ARG DEBIAN_FRONTEND=noninteractive
 ---> Using cache
 ---> 88306a3cab77
Step 3/20 : SHELL [ "/bin/bash","-c" ]
 ---> Using cache
 ---> 823a73990dba
Step 4/20 : ENV NVIDIA_VISIBLE_DEVICES all
 ---> Using cache
 ---> 72cf8e27e0ab
Step 5/20 : ENV NVIDIA_DRIVER_CAPABILITIES compute,video,utility
 ---> Using cache
 ---> 9490e3109300
Step 6/20 : RUN apt update -y && apt upgrade -y
 ---> Using cache
 ---> 969f84ed6f63
Step 7/20 : RUN apt install wget -y && apt install git -y && apt install libaio-dev -y && apt install libaio1 -y
 ---> Using cache
 ---> f42adf5187a7
Step 8/20 : RUN apt install python3.9 -y && apt install python3-pip -y && apt install python-is-python3 -y
 ---> Using cache
 ---> 896beed91d20
Step 9/20 : RUN pip install torch torchvision torchaudio
 ---> Using cache
 ---> e18593d3bfb5
Step 10/20 : RUN pip install datasets && pip ins

WARNING! Your password will be stored unencrypted in /home/ec2-user/.docker/config.json.
Configure a credential helper to remove this warning. See
https://docs.docker.com/engine/reference/commandline/login/#credentials-store



## SageMaker Training

Once the image has been pushed to ECR we can then kick off the Training Job but first we need to create a SageMaker Estimator object. That contains the information required to start a Training Job. 

There is a number of paramaters you can tune. Depending on the number of GPUs avaialble you set `num_gpus`. The deepspeed configratuon file is also parameterized. 

There are three options to choose from:
1. ds_config_stage1.json
2. ds_config_stage2.json
3. ds_config_stage3.json

https://github.com/mallorbc/Finetune_GPTNEO_GPTJ6B/tree/main/finetuning_repo#deepspeed


Training and finetuning a model is an experimental science. You may want to tune different learning rates, weight decay, etc.

The Training Job has also been configured to emit metrics such as `eval_loss`. The regex for the metrics that the container emits is specified in `metric_definitions`.  You can use these metrics to decide if you wish to stop the Training Job if you do not see an improvement in the loss.



In [20]:
import boto3
import sagemaker 
from sagemaker import get_execution_role
from sagemaker.predictor import csv_serializer

from sagemaker import local

sagemaker_session = sagemaker.Session()

#local_sagemaker_session = local.LocalSession()

role = get_execution_role()

account = sagemaker_session.boto_session.client('sts').get_caller_identity()['Account']
region = sagemaker_session.boto_session.region_name

image = '{}.dkr.ecr.{}.amazonaws.com/gptj-finetune:latest'.format(account, region)

bucket = sagemaker_session.default_bucket() # Set a default S3 bucket
prefix = 'DEMO-fine-tune-GPTJ'


sm_model = sagemaker.estimator.Estimator(
image_uri=image,
role=role,
instance_count = 1,
#instance_type='local_gpu', 
#sagemaker_session=local_sagemaker_session,
sagemaker_session= sagemaker_session,
#instance_type = 'ml.g5.48xlarge',
instance_type="ml.g5.12xlarge",
environment = {
    "num_gpus": "4",
    "deepspeed": "ds_config_stage3.json",
    "evaluation_strategy": "steps",
    "output_dir": "/opt/ml/model/finetune",
    "num_train_epochs": "12",
    "eval_steps": "20",
    "gradient_accumulation_steps": "1",
    "per_device_train_batch_size": "4",
    "use_fast_tokenizer": "False",
    "learning_rate": "5e-06",
    "warmup_steps": "10",
    "save_total_limit": "1",
    "save_steps": "20",
    "save_strategy": "steps",
    "tokenizer_name": "gpt2",
    "load_best_model_at_end": "True",
    "block_size": "2048",
    "weight_decay": "0.1"
},
    
output_path=f"s3://{bucket}/fine-tune-GPTJ/output/",

metric_definitions=[
    {'Name': 'eval:loss', 'Regex': "'eval_loss': ([0-9]+\.[0-9]+)"},
    {'Name': 'eval:runtime', 'Regex': "'eval_runtime': ([0-9]+\.[0-9]+)"},
    {'Name': 'eval:samples_per_second', 'Regex': "'eval_samples_per_second': ([0-9]+\.[0-9]+)"},
    {'Name': 'eval:eval_steps_per_second', 'Regex': "'eval_steps_per_second': ([0-9]+\.[0-9]+)"},
]

)


INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole
INFO:botocore.credentials:Found credentials from IAM Role: BaseNotebookInstanceEc2InstanceRole


In our case we will be using the processed quotes dataset to finetune GPTJ and as such we upload the train and validation set to S3. 

In [21]:
train_s3=f"s3://{bucket}/fine-tune-GPTJ/datasets/train/train.csv"
val_s3=f"s3://{bucket}/fine-tune-GPTJ/datasets/validation/validation.csv"


!aws s3 cp ../Finetune_GPTNEO_GPTJ6B/quotes_dataset/train.csv $train_s3
!aws s3 cp ../Finetune_GPTNEO_GPTJ6B/quotes_dataset/validation.csv $val_s3


upload: ../Finetune_GPTNEO_GPTJ6B/quotes_dataset/train.csv to s3://sagemaker-us-east-1-171503325295/fine-tune-GPTJ/datasets/train/train.csv
upload: ../Finetune_GPTNEO_GPTJ6B/quotes_dataset/validation.csv to s3://sagemaker-us-east-1-171503325295/fine-tune-GPTJ/datasets/validation/validation.csv


The container we bult expects two datasets, a train and validation dataset. Here set the training input channels "train" and "validation" which each point to their respective S3 locations for SageMaker to make use of during Training. SageMaker handles downloading the datasets to the container from S3 on our behalf. 

Finally, we kick off the job with the `.fit()` method. 

In [ ]:
from sagemaker.session import TrainingInput

train_input = TrainingInput(
    train_s3, content_type="csv"
)
validation_input = TrainingInput(
    val_s3, content_type="csv"
)

sm_model.fit({"train": train_input, "validation": validation_input}, wait=True)


INFO:sagemaker:Creating training-job with name: gptj-finetune-2023-02-09-06-35-31-223


2023-02-09 06:35:31 Starting - Starting the training job...
2023-02-09 06:35:59 Starting - Preparing the instances for training......
2023-02-09 06:37:00 Downloading - Downloading input data...
2023-02-09 06:37:15 Training - Downloading the training image.....................
2023-02-09 06:40:41 Training - Training image download completed. Training in progress...Filesystem      Size  Used Avail Use% Mounted on
overlay          65G   35G   31G  54% /
tmpfs            64M     0   64M   0% /dev
tmpfs            94G     0   94G   0% /sys/fs/cgroup
/dev/nvme0n1p1   65G   35G   31G  54% /usr/local/nvidia
/dev/nvme2n1    3.5T  8.5M  3.3T   1% /tmp
shm              93G     0   93G   0% /dev/shm
tmpfs            94G   12K   94G   1% /proc/driver/nvidia
tmpfs            94G  4.0K   94G   1% /etc/nvidia/nvidia-application-profiles-rc.d
tmpfs            94G  1.4M   94G   1% /run/nvidia-persistenced/socket
devtmpfs         94G     0   94G   0% /dev/nvidia0
tmpfs            94G     0   94G   0% /pr